In [15]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
import os
import xgboost as xgb
import matplotlib.pyplot as plt


In [45]:
all_data=pd.read_csv("axis2_demo_tablepart1_0.csv")

print(len(all_data))

#data = pd.read_csv('axis2_demo_tablepart1_0.csv')

# Assuming 'important_features.json' contains a JSON array of feature names
import json

with open('important_features.json', 'r') as file:
    features = json.load(file)[:10]

data_selected = all_data[features]

10000


In [46]:
data_selected.head(5)

,Standardabweichung_y,Variance_z,Variance_y,Standardabweichung_z,Effektivwert_x,Mittlere_Absolute_Abweichung_y,Effektivwert_y,Standardabweichung_x,Mittelwert_x,Variance_x
0,163.966,220962,26911.8,469.831,657.526,117.401,571.115,328.477,569.600,108005.0
1,171.940,235057,29592.9,484.584,627.592,122.691,571.746,309.810,545.792,96078.5
2,156.322,256573,24460.9,506.277,651.770,111.817,566.507,324.498,565.248,105404.0
3,174.714,241982,30555.6,491.670,653.578,123.620,568.931,325.453,566.784,106026.0
4,162.434,268509,26411.1,517.919,661.006,128.562,586.402,339.697,567.040,115510.0


In [55]:
scaler = MinMaxScaler(feature_range=(0, 1))
data_scaled = scaler.fit_transform(data_selected)
data_scaled[2:3 , :]

array([[1.10611154e-03, 4.36220241e-04, 2.06424099e-05, 6.25969959e-03,
        2.72064937e-02, 1.59843147e-03, 1.64782844e-02, 3.81172612e-03,
        6.84932035e-01, 1.89942632e-04]])

In [37]:

# 数据归一化
scaler = MinMaxScaler(feature_range=(0, 1))
data_scaled = scaler.fit_transform(data_selected)


# 准备数据集
def create_dataset(dataset, look_back=1):
    X, Y = [], []
    for i in range(len(dataset) - look_back):
        a = dataset[i:(i + look_back), :]
        X.append(a)
        b = dataset[i + look_back, :]
        Y.append(b)
    return np.array(X), np.array(Y)

look_back = 1
X, Y = create_dataset(data_scaled, look_back)
X_train, X_test = X[:9000], X[9000:]
Y_train, Y_test = Y[:9000], Y[9000:]
X_train=X_train.reshape(len(X_train),len(features))
X_test=X_test.reshape(len(X_test),len(features))
# 使用XGBoost训练模型
model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100)
model.fit(X_train, Y_train)

# 进行预测
y_pred = model.predict(X_test)

# print the actual and predicted values for each feature
import matplotlib.pyplot as plt

true_test = scaler.inverse_transform(Y_test)
y_pred_test = scaler.inverse_transform(y_pred)
# 创建足够的子图，每个特征一个子图，所有子图排列成一列
fig, axs = plt.subplots(len(features), 1, figsize=(10, 2*len(features)))  # 高度根据特征数量动态调整

for i in range(len(features)):
    # 如果只有一列，当只有一个特征时，axs不是列表，需要做一下处理
    ax = axs[i] if len(features) > 1 else axs
    
    ax.plot(true_test[:, i], label='Actual')
    ax.plot(y_pred_test[:, i], label='Predicted')
    ax.set_title(f'Feature: {features[i]}')
    ax.legend()

plt.tight_layout()  # 自动调整子图参数，使之填充整个图像区域
plt.show()


In [68]:
import numpy as np

# 生成一个6x10的随机数组
array = np.random.rand(1, 10)

print(array)


[[0.48015173 0.38820822 0.23737739 0.46326693 0.89850229 0.69953667
  0.854714   0.34200662 0.29334272 0.9673981 ]]


In [67]:
array[1].shape

(10,)

In [69]:
y_pred_arr = model.predict(array)

y_pred_arr


array([[0.66890293, 0.22877522, 0.6208459 , 0.42756167, 0.34186634,
        0.7397818 , 0.50888264, 0.15853034, 0.4799016 , 0.05330889]],
      dtype=float32)